In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings

from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from keras.models import Model

warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
dataset = pd.read_csv('data/ratings.csv')

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(dataset, test_size=0.2, random_state=42)

Most recommendation systems are build using a simple dot product as shown below but newer ones are now implementing a neural network instead of the simple dot product.

In [ ]:
book_input = Input(shape=[1], name="Book-Input")
book_embedding = Embedding(n_books+1, 5, name="Book-Embedding")(book_input)
book_vec = Flatten(name="Flatten-Books")(book_embedding)

# creating user embedding path
user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users+1, 5, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)

# performing dot product and creating model
prod = Dot(name="Dot-Product", axes=1)([book_vec, user_vec])
model = Model([user_input, book_input], prod)
model.compile('adam', 'mean_squared_error')

In [ ]:
from keras.models import load_model

if os.path.exists('regression_model.h5'):
    model = load_model('regression_model.h5')
else:
    history = model.fit([train.user_id, train.book_id], train.rating, epochs=5, verbose=1)
    model.save('regression_model.h5')
    plt.plot(history.history['loss'])
    plt.xlabel("Epochs")
    plt.ylabel("Training Error")

In [ ]:
model.evaluate([test.user_id, test.book_id], test.rating)

In [ ]:
predictions = model.predict([test.user_id.head(10), test.book_id.head(10)])

[print(predictions[i], test.rating.iloc[i]) for i in range(0,10)]

Neural Networks proved there effectivness for almost every machine learning problem as of now and they also perform exceptionally well for recommendation systems.

In [ ]:
book_input = Input(shape=[1], name="Book-Input")
book_embedding = Embedding(n_books+1, 5, name="Book-Embedding")(book_input)
book_vec = Flatten(name="Flatten-Books")(book_embedding)

# creating user embedding path
user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users+1, 5, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)

# concatenate features
conc = Concatenate()([book_vec, user_vec])

# add fully-connected-layers
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
out = Dense(1)(fc2)

# Create model and compile it
model2 = Model([user_input, book_input], out)
model2.compile('adam', 'mean_squared_error')

In [ ]:
from keras.models import load_model

if os.path.exists('regression_model2.h5'):
    model2 = load_model('regression_model2.h5')
else:
    history = model2.fit([train.user_id, train.book_id], train.rating, epochs=5, verbose=1)
    model2.save('regression_model2.h5')
    plt.plot(history.history['loss'])
    plt.xlabel("Epochs")
    plt.ylabel("Training Error")

In [ ]:
model2.evaluate([test.user_id, test.book_id], test.rating)

In [ ]:
predictions = model2.predict([test.user_id.head(10), test.book_id.head(10)])

[print(predictions[i], test.rating.iloc[i]) for i in range(0,10)]

### Results & further improvements

    1) We can see improvements ,better than traditional collaborative filtering methods
    2) Our deep learning is better as collaborative filtering is not a suitable model to deal with cold start problem, in which it cannot draw any inference for users or items about which it has not yet gathered sufficient information.
    3) We can also different deep learning architechture like encoder-decoder architechture , which me further improve our results.



### How does bias impact search and recommender systems?

Well, most web systems are optimized by using implicit user feedback. However, user data is partly biased by the choices that these systems make.

For example, we can only click on things that are shown to us.

Because these systems are usually based on Machine Learning, they learn to reinforce their own biases, yielding self-fulfilled prophecies and/or sub-optimal solutions.

For example, personalization and filter bubbles for users can create echo chambers for recommender systems.

In addition, these systems sometimes compete among themselves. So, an improvement in one system (e.g., user experience) might be just a degradation in another system (e.g., monetization) that uses a different (even inversely correlated) optimization function.


### How can we overcome biases?

    1) Data
    Analyze for known and unknown biases, debias, or mitigate when possible/needed.
    Recollect more data for difficult/sparse regions of the problem.
    Delete attributes associated directly/indirectly with harmful bias.

    2) Interaction
    Make sure that the user is aware of the biases all the time.
    Give more control to the user.

### Evaluation metrics

    1) Recall as we don't want to miss people and want to catch correct recommendation for them
    2) Good Sensitivity and Specificity 

### Model monitoring

    1) Keep a check on recall or RMSE regulary and retrain the model as soon as increase of RMSE or drop of recall
    2) Prevent data leakage , by splitting the data using optimum startegy
    3) Add more data and split according keep in mind addition of new users 